In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### Q1. Create Model to predict the Telco customer churn (Churn = Label).
<b><font color=red>The difference of Train and Test accuracy must be lower than 10% to get full score.</font></b>

In [2]:
df = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

In [3]:
df.shape

In [4]:
df.isnull().any() #เช็ค missing data

In [5]:
df.isnull().sum(axis=0)

In [9]:
df_clean = df

In [10]:
df_clean = df_clean.drop(['PaperlessBilling', 'Contract', 'tenure','Dependents', 'SeniorCitizen'],axis = 1)
df_clean.isnull().sum(axis=0)

In [11]:
df_clean = df_clean.drop(['Partner'],axis = 1)
df_clean.isnull().sum(axis=0)

In [12]:
df_clean.dtypes

In [14]:
#แปลงข้อมูลเป็นตัวเลข
df_clean['customerID'] = pd.factorize(df_clean['customerID'])[0]
df_clean['gender'] = pd.factorize(df_clean['gender'])[0]
df_clean['PhoneService'] = pd.factorize(df_clean['PhoneService'])[0]

df_clean['MultipleLines'] = pd.factorize(df_clean['MultipleLines'])[0]
df_clean['InternetService'] = pd.factorize(df_clean['InternetService'])[0]
df_clean['OnlineSecurity'] = pd.factorize(df_clean['OnlineSecurity'])[0]

df_clean['OnlineBackup'] = pd.factorize(df_clean['OnlineBackup'])[0]
df_clean['DeviceProtection'] = pd.factorize(df_clean['DeviceProtection'])[0]
df_clean['TechSupport'] = pd.factorize(df_clean['TechSupport'])[0]

df_clean['StreamingTV'] = pd.factorize(df_clean['StreamingTV'])[0]
df_clean['StreamingMovies'] = pd.factorize(df_clean['StreamingMovies'])[0]
df_clean['PaymentMethod'] = pd.factorize(df_clean['PaymentMethod'])[0]

df_clean['MonthlyCharges'] = pd.factorize(df_clean['MonthlyCharges'])[0]
df_clean['TotalCharges'] = pd.factorize(df_clean['TotalCharges'])[0]
df_clean['Churn'] = pd.factorize(df_clean['Churn'])[0]

df_clean


In [15]:
df_clean.dtypes

In [17]:
X = df_clean.drop('Churn', axis = 1)
y = df_clean['Churn']

In [18]:
y

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [23]:
X_train.shape

In [24]:
X_test.shape

In [25]:
y_train.shape

In [26]:
y_test.shape

In [27]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [45]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X_train, y_train)

params = {'C' : [1, 2, 4, 8, 16]}

from sklearn.pipeline import Pipeline
clf = Pipeline([
  ('scaler', MinMaxScaler(feature_range=(0, 1))),
  ('feature_selection', SelectKBest(f_classif, k=5)),
  ('classification', GridSearchCV(LogisticRegression(random_state=0, solver='liblinear'),params, cv=5))
])
clf.fit(X_res, y_res)

In [46]:
acc = best_clf.best_score_
print("10CV accuracy: "+str(acc))

In [47]:
best_clf.best_params_

In [48]:
yp = best_clf.predict(X_test)
acc = sum(yp == y_test)/len(y_test)
print("Test Training accuracy: "+str(acc))

In [49]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
target_names = ['negative(0)', 'positive(1)']
C = confusion_matrix(y_test,yp) 
C = C / C.astype(np.float).sum(axis=1)*100
sns.heatmap(C, annot=True, fmt="f",cmap="GnBu",xticklabels=target_names, yticklabels=target_names)
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()